<a href="https://colab.research.google.com/github/Redislabs-Solution-Architects/Redis-Workshops/blob/main/02-Vector_Similarity_Search/02.02_Redis_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vector Similarity Search with Redis

[Always-on demo](https://antonum-redis-vss-streamlit-streamlit-app-p4z5th.streamlit.app/)

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

This notebook generates vector embeddings using pre-trained `sentence-transformers/all-MiniLM-L6-v2` model from HuggingFace, loads them to Redis and runs Vector Similarity search against Redis database.

In [1]:
#install Redis client and Hugging Face sentence transformers
!pip install -q redis sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.3/261.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 7.9 MB/s eta 0:00:00


Install Redis Stack locally

In [2]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes


deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


### Connect to the Redis server

In [3]:
import os

REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
#Replace values above with your own if using Redis Cloud instance
#REDIS_HOST="redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
#REDIS_PORT=18374
#REDIS_PASSWORD="1TNxTEdYRDgIDKM2gDfasupCADXXXX"

#shortcut for redis-cli $REDIS_CONN command
# If SSL is enabled on the endpoint add --tls
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"
INDEX_NAME = f"qna:idx"

# Test Redis connection
!redis-cli $REDIS_CONN PING

PONG


In [4]:
import redis
redis = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD)
redis.ping()

True

In [5]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
from redis import Redis
from redis.commands.search.field import (
    NumericField,
    TagField,
    TextField,
    VectorField,
)
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query


tqdm.pandas()



/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


### Embedding generation model

Here we are using `sentence-transformers/all-MiniLM-L6-v2` from HuggingFace. https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2



In [6]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Download 12k+ tweets

In [7]:
!wget https://raw.githubusercontent.com/antonum/Redis-VSS-Streamlit/main/Labelled_Tweets.csv

--2024-10-05 17:21:22--  https://raw.githubusercontent.com/antonum/Redis-VSS-Streamlit/main/Labelled_Tweets.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2486081 (2.4M) [text/plain]
Saving to: ‘Labelled_Tweets.csv’

Labelled_Tweets.csv 100%[===================>]   2.37M  --.-KB/s    in 0.06s   

2024-10-05 17:21:23 (41.9 MB/s) - ‘Labelled_Tweets.csv’ saved [2486081/2486081]



In [8]:
df = pd.read_csv('Labelled_Tweets.csv').drop(columns=['created_at','score'])
#df=df.head(3000) #trim dataframe to fit results into 30MB Redis database
df


,id,full_text
0,1,@KennyDegu very very little volume. With $10T ...
1,2,#ES_F achieved Target 2780 closing above 50% #...
2,3,RT @KimbleCharting: Silver/Gold indicator crea...
3,4,@Issaquahfunds Hedged our $MSFT position into ...
4,5,RT @zipillinois: 3 Surprisingly Controversial ...
...,...,...
12415,12587,RT @PeterLBrandt: $SPX $ES_F \r\nFollowing thi...
12416,12588,RT @vieiraUAE: Fearless Alex Vieira Calls Best...
12417,12589,$spy $spx $qqq $ndx #nyse going from poking th...
12418,12590,RT @DavidScottAdams: On watch tomorrow // Pt. ...


### Generate Embeddings

Generate vector embeddings within the dataframe. This step can take 2-3 minutes (unoptimized version) and 10 seconds (optimized version) on GPU runtime for all 12k records.

In [9]:
# Unoptimized version, processing one embedding at a time
#def text_to_embedding(text):
#  return model.encode(text).astype(np.float32).tobytes()

# Optimized version with for parallel processing
def texts_to_embeddings(texts):
  return [np.array(embedding, dtype=np.float32).tobytes() for embedding in model.encode(texts, show_progress_bar=True)]


#generate vector embeddings
#df["text_embedding"] = df["full_text"].progress_apply(text_to_embedding)
df["text_embedding"] = texts_to_embeddings(df["full_text"].tolist())
df

Batches:   0%|          | 0/389 [00:00<?, ?it/s]

,id,full_text,text_embedding
0,1,@KennyDegu very very little volume. With $10T ...,b')\x92\x81\xbd\x1fh\x8b\xbdl\xdf\xe4\xbc\xc2\...
1,2,#ES_F achieved Target 2780 closing above 50% #...,b']\x1b\x02\xbd\x19~/\xbd\x91z\xb1\xbcQ\x99\xd...
2,3,RT @KimbleCharting: Silver/Gold indicator crea...,b'\x10\xaa\xa3\xbdi}\x10\xbd\xc5\xe8\xb9=%\x08...
3,4,@Issaquahfunds Hedged our $MSFT position into ...,"b""\xde\x7f\xd1\xbc\xc5\n`\xbd>9 =\xe6\xc0\xef=..."
4,5,RT @zipillinois: 3 Surprisingly Controversial ...,b'\xd8\r\x1e\xbd:\\\xd4\xbcf/\xa1\xbc\xe7q7=\x...
...,...,...,...
12415,12587,RT @PeterLBrandt: $SPX $ES_F \r\nFollowing thi...,b'\x03y\x87\xbbT\x13\x823\x03\x02*\xbd!\x9f\xe...
12416,12588,RT @vieiraUAE: Fearless Alex Vieira Calls Best...,b'$\xeb><\xb5\xf7\x97=\xdf\xe3\x12\xbd\x8c\xac...
12417,12589,$spy $spx $qqq $ndx #nyse going from poking th...,b'\x0e\xef\xa1\xbd!\xf8\x8e\xbdU_\xa6=>\x0c\x9...
12418,12590,RT @DavidScottAdams: On watch tomorrow // Pt. ...,b'\xb7\xba\xb1\xbdo-)=\xba\xb5}\xbc\x12\x1eK=y...


### Create Helper Functions

- Save dataframe to Redis HASH
- Create RediSearch Index

In [10]:
def load_dataframe(redis, df, key_prefix="tweet", id_column="id", pipe_size=100):
    records = df.to_dict(orient="records")
    pipe = redis.pipeline(transaction=False)
    i=1
    for record in tqdm(records):
        i=i+1
        key = f"{key_prefix}:{record[id_column]}"
        pipe.hset(key, mapping=record)
        if (i+1) % pipe_size == 0:
          res=pipe.execute()
    pipe.execute()

def create_redis_index(redis, idxname="tweet:idx"):
  try:
    redis.ft(idxname).dropindex()
  except:
    print("no index found")

  # Create an index
  indexDefinition = IndexDefinition(
      prefix=["tweet:"],
      index_type=IndexType.HASH,
  )

  redis.ft(idxname).create_index(
      (
          TextField("full_text", no_stem=False, sortable=False),
          VectorField("text_embedding", "HNSW", {  "TYPE": "FLOAT32",
                                                    "DIM": 384,
                                                    "DISTANCE_METRIC": "COSINE",
                                                  })
      ),
      definition=indexDefinition
  )

### Create index and load data to Redis

In [11]:
# clear Redis database (optional)
redis.flushdb()

# create Index
create_redis_index(redis)

# load data from Dataframe to Redis HASH
load_dataframe(redis,df,key_prefix="tweet", pipe_size=100)


no index found


  0%|          | 0/12420 [00:00<?, ?it/s]

In [12]:
#Check how the data is stored in Redis
!redis-cli $REDIS_CONN hgetall "tweet:1001"

1) "full_text"
2) "1900 PIPS Profit so far\xf0\x9f\x94\xa5\xf0\x9f\x94\xa5\r\n\r\nFor free signals, Join https://t.co/eRcU1NYwV1\r\n\r\n#bitcoin #Forex #amzn #fx $BTC #MT5 #MT4 #xagusd #GBPAUD #ForexAnalysis #nflx #ForexGroup #gbpjpy #forexsignals #stocks #METATRADER #usoil #forexnews 68016 https://t.co/WUP4laPFlT"
3) "id"
4) "1001"
5) "text_embedding"
6) "G\x1e\x03:\xc9b\xbd\xbc5\xb0\x94\xbc\x1cK)< U\x94;\x19>\x11\xbd\xae3\r=\x03\x98\x7f=\x9eX\x88<tUR\xbd\xc8\xf6h\xbcH\xb5\xd2\xbb\x0e;\t\xbd\xc1m#<\xdf\xd4C=\x98\xc8\"\xbd\x1f\xa3-\xbd1^\t\xbdb\x00\xfd\xbc\xb2GE\xbd@\xadZ\xbc]\xae=\xbd;<\x99<\xf7b\xce\xbc\xff_\xab<\"DT\xbcpM\xd7\xbc\x8e\xf0\x82\xbb\x8d\xb0(=5v\x8a\xbc\xc7\xa8M\xbd],P=&S\x86<\xa4j\xfa<>?\xb5<\x93[\xee\xbb\xcd\x8d\x90=\bW8=Q\x81\x82\xbd\xab\xfb&<\xf7-:=S\xe6\xe4\xbd\xfd\x1d\x11\xbc\xce\xb7\xac\xbd>Z<\xbbp\xc9\xcc\xbb\xd39b\xbcF\xe3\x90=*\x9f\xa7=\xf57\x0f=x\xbb\xbb\xbbS\xad\x17=\xb6\xaeE\xbd\xdc\x8e\x90=v\xd7\x02\xbd\xd8\x01\x9f\xbc\xe1\xc3e;\n\xa5\xd8\xbcg{f<\x9ax\xaf\x

## Query the database

[Alway-on Streamlit app](https://antonum-redis-vss-streamlit-streamlit-app-p4z5th.streamlit.app/)


Try queries like:
“Oil”, “Oil Reserves”, “Fossil fuels”

Lexical Full Text search quickly runs out of matches

Vector search continues to discover relevant tweets

In [13]:
user_query="oil price"
# queries to try "oil reserve", "fossil fuels"

In [14]:
#using Full Text Index
q = Query(user_query)\
  .return_fields("full_text")
res = redis.ft("tweet:idx").search(q)
if res.total==0:
  print("No matches found")
else:
  res_df = pd.DataFrame([t.__dict__ for t in res.docs ]).drop(columns=["payload"])
  display(res_df)

,id,full_text
0,tweet:3220,The relative performance of TIPS has historica...
1,tweet:311,"Oil Prices Rise, Fall As Russia, Saudi Arabia ..."
2,tweet:1490,"RT @Benzinga: Oil Prices Rise, Fall As Russia,..."
3,tweet:1585,"RT @Benzinga: Oil Prices Rise, Fall As Russia,..."
4,tweet:1610,"Oil Prices Rise, Fall As Russia, Saudi Arabia ..."
5,tweet:7189,Do higher oil prices help the consumer and sma...
6,tweet:636,Told you Saudi Arabia will bend the knee @jimc...
7,tweet:5405,https://t.co/3IJBXa5wuf Historic oil price plu...
8,tweet:5406,Historic oil price plunge trashes sector's pro...
9,tweet:3865,Today's book recommendation goes for the winne...


In [15]:
# Using Vector Similarity Index
#query_vector=text_to_embedding(user_query)
query_vector=texts_to_embeddings([user_query])[0]
q = Query("*=>[KNN 10 @text_embedding $vector AS result_score]")\
                .return_fields("result_score","full_text")\
                .dialect(2)\
                .sort_by("result_score", True)
res = redis.ft("tweet:idx").search(q, query_params={"vector": query_vector})
#print(res)
res_df = pd.DataFrame([t.__dict__ for t in res.docs ]).drop(columns=["payload"])
res_df

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,id,result_score,full_text
0,tweet:444,0.369450807571,Would you spend $2 more a gallon of gasoline i...
1,tweet:11529,0.371095538139,RT @tradingcrudeoil: Crude oil closed up $0.48...
2,tweet:5654,0.381934285164,..and oil still 25.74 LMAO &gt;&gt;&gt;NO DEM...
3,tweet:204,0.396132171154,Bad news for #oil. It’s going to between $10 ...
4,tweet:9189,0.409308671951,Oil erases gains for the day in fall to $25 ht...
5,tweet:7189,0.42981672287,Do higher oil prices help the consumer and sma...
6,tweet:9330,0.430081129074,The price of Texas intermediate oil (WTI) slum...
7,tweet:531,0.431391477585,#OIL Sentiment ($22.50)\r\n\r\nWhat’s next for...
8,tweet:1490,0.441061556339,"RT @Benzinga: Oil Prices Rise, Fall As Russia,..."
9,tweet:1585,0.441061556339,"RT @Benzinga: Oil Prices Rise, Fall As Russia,..."
